In [145]:
using Plots
using Base.Threads
using CUDA
using DelimitedFiles

import LinearAlgebra

In [146]:
function nedft(x :: Array{Float64, 1}, F :: Array{ComplexF64, 1})
    N = length(F)
    if length(x) != N
        error("Length mismatch")
    end
    
    f = zeros(ComplexF64, N)

    for j = 1:N
        for k = 1:N
            @inbounds f[j] += F[k] * exp(-2*π*1im*x[j]*(k-1-N/2))
        end
    end

    f
end

nedft (generic function with 1 method)

In [147]:
function nedft_simpl(x :: Array{Float64, 1}, F :: Array{ComplexF64, 1})
    N = length(F)
    if length(x) != N
        error("Length mismatch")
    end
    
    f = zeros(ComplexF64, N)

    for j = 1:N
        e_j = exp(-2*π*1im*x[j])^(-N/2)
        for k = N:-1:1
            @inbounds f[j] *= e_j
            @inbounds f[j] += F[k]
        end
    end

    f
end

nedft_simpl (generic function with 1 method)

In [148]:
function nedft_outer_threaded(x :: Array{Float64, 1}, F :: Array{ComplexF64, 1})
    N = length(F)
    if length(x) != N
        error("Length mismatch")
    end
    
    f = zeros(ComplexF64, N)

    @threads for j = 1:N
        for k = 1:N
            @inbounds f[j] += F[k] * exp(-2*π*1im*x[j]*(k-1-N/2))
        end
    end

    f
end

nedft_outer_threaded (generic function with 1 method)

In [149]:
# GEFAHR --- Da Threads asynchron funktioniert Horner-style Multiplikation nicht, wenn man die k's aufteilt!!!
function nedft_inner_threaded(x :: Array{Float64, 1}, F :: Array{ComplexF64, 1})
    N = length(F)
    if length(x) != N
        error("Length mismatch")
    end
    
    f = zeros(ComplexF64, N)

    for j = 1:N
        @threads for k = 1:N
            @inbounds f[j] += F[k] * exp(-2*π*1im*x[j]*(k-1-N/2))
        end
    end

    f
end

nedft_inner_threaded (generic function with 1 method)

In [150]:
function nedft_horner_threaded(x :: Array{Float64, 1}, F :: Array{ComplexF64, 1})
    N = length(F)
    
    if length(x) != N
        error("Length mismatch")
    end
    
    f :: Array{ComplexF64, 1} = zeros(ComplexF64, N)

    @threads for j = 1:N
        e_j = exp(-2*π*1im*x[j])
        for k = N:-1:1
            @inbounds f[j] = f[j] * e_j + F[k]
        end
        f[j] = f[j] * e_j^(-N/2) 
    end

    f
end

nedft_horner_threaded (generic function with 1 method)

In [151]:
function nedft_cuda(x :: Array{Float64, 1}, F :: Array{ComplexF64, 1})
    N = length(x)
    
    # Copy data to GPU
    F_d = CuArray(F)
    x_d = CuArray(x)
    f_d = CUDA.zeros(ComplexF64, N)
    e_d = CUDA.zeros(Float64, N)

    CUDA.copyto!(e_d, x_d)
    
    s = -2*π*1im
    
    e_d = broadcast(CUDA.exp, s.*e_d)

    for k = N:-1:1
        CUDA.fill!(F_d, F[k])
        f_d .= f_d .* e_d .+ F_d
    end
    CUDA.copyto!(e_d, x_d)
    e_d = broadcast(CUDA.exp, (-N/2)*s.*e_d)
    f_d =  f_d .* e_d

    # f_d von GPU 
    Array(f_d)
end

nedft_cuda (generic function with 1 method)

In [152]:
# Precompile funcs
@time begin
    local N = 2^2                        # Length of input vector
    local F = rand(ComplexF64, N)        # Fourier Coefficients
    local x = rand(Float64, N) .- 0.5    # Sample points (-.5, .5]
    
    nedft(x,F)
    nedft_simpl(x, F)
    nedft_basic_threaded(x, F)
    nedft_horner_threaded(x, F)
    nedft_cuda(x, F)                     # Vorallem hier viel Overhead
end;

  0.069708 seconds (65.53 k allocations: 3.498 MiB)


In [154]:
max_exp = 17
data = zeros(6,max_exp)

for exponent = 1:max_exp
    local N = 2^exponent                 # Length of input vector
    local F = rand(ComplexF64, N)        # Fourier Coefficients
    local x = rand(Float64, N) .- 0.5    # Sample points (-.5, .5]

    data[1, exponent] = @elapsed nedft(x,F)
    data[2, exponent] = @elapsed nedft_simpl(x, F)
    data[3, exponent] = @elapsed nedft_outer_threaded(x, F)
    data[4, exponent] = @elapsed nedft_horner_threaded(x, F)
    data[5, exponent] = @elapsed nedft_cuda(x, F)
end

<img src="vgl1.png">

In [163]:
Plots.plot( data[1, :], label="standard", title="Performance of various implementations", xlabel="Time in s", size=(1200,600))
plot!(data[2, :], label="horner")
plot!(data[3, :], label="threads ($(nthreads()))")
plot!(data[4, :], label="threads ($(nthreads()))+horner")
plot!(data[5, :], label="CUDA")

xlabel!("Length of Input (log_2)")
ylabel!("Time in seconds");

<img src="benchmark2.png">

In [171]:
Plots.plot( data[1, 1:5], label="standard", title="Performance of various implementations", xlabel="Time in s", size=(1200,600))
plot!(data[2, 1:5], label="horner")
plot!(data[3, 1:5], label="threads ($(nthreads()))")
plot!(data[4, 1:5], label="threads ($(nthreads()))+horner")
plot!(data[5, 1:5], label="CUDA")

xlabel!("Length of Input (log_2)");

<img src="benchmark3.png">

In [175]:
Plots.plot( data[1, 1:5], label="standard", title="Performance of various implementations", xlabel="Time in s", size=(1200,600))
plot!(data[2, 1:5], label="horner")
plot!(data[3, 1:5], label="threads ($(nthreads()))")
plot!(data[4, 1:5], label="threads ($(nthreads()))+horner")
xlabel!("Length of Input (log_2)");

<img src="benchmark4.png">